File: prices.ipynb\
Author: alexkobz\
Date: 08-10-2024\
Task: DSRFU-298

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = last_day_month
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [2]:
from rudata.RuDataDF import RuDataDF
from rudata.RuDataRequest import RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


In [3]:
month_year = REPORT_DATE.strftime("%m.%Y")
isin_file = f"ISIN_{month_year}.xlsx"
Prices = pd.read_excel(f"./data/Input/{isin_file}", names=["ISIN"]).dropna().set_index("ISIN")
Prices["Date"] = REPORT_DATE
Prices[["Name", "Type", "Principal", "Currency", "Clean price pct", "Clean price", "Interest", "Price", 
        "Source", "ISIN in DB", "FX Rate", "RUB Price", "CBR discount 6D", "Discount NCC", "NCC collateral"]] = np.nan

#### Шаг 1. RuData

##### Шаг 1.0. RuData stage

In [4]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
FintoolReferenceData finished. FintoolReferenceData shape (102155, 186)


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,2.0,2.0,2.0,2.0,2.0,None,2.0,0.0,2.0,2.0,2.0,2.0,2.0,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,S111,None,Старшие необеспеченные,4000,None,None,"МинФин РФ, государственные краткосрочные беску...",None,None,ГКО-21134,NaN,21134RMFS,None,МинФин,0.0,act/365,None,None,Дисконтный,Облигация,None,Необеспеченные,0.0,0.0,0.0,Краткосрочные,None,1.0,RU,0.0,RUB,1000.0,NaN,0.0,0.0,Погашен,2.500000e+09,2500000.0,2.500000e+09,2500000.0,Минфин РФ,00013474,Государственные,6874.0,7710168360,RU,г.Москва,Минфин РФ,00013474,Государственные,6874.0,7710168360,RU,г.Москва,None,0.0,1999-12-17T00:00:00,None,110.0,1999-12-17T00:00:00,2000-04-05T00:00:00,0.0,1999-12-17T00:00:00,Гос,Классические,Постоянный,2023-04-19T15:55:00,NaN,Нет информации,0.0,None,NaN,None,NaN,False,False,False,2.0,NaN,True,None,0.0,False,False,None,RU,1000,1000,NaN,False,NaN,False,False,None,416.0,NaN,None,None,102155,1
1,3.0,3.0,3.0,4.0,3.0,SU21135RMFS7,3.0,1.0,2.0,3.0,2.0,3.0,2.0,Государственные краткосрочные бескупонные обли...,Финансовые органы,None,135,0.0,0.0,None,0.0,0.0,1000.0,RUB,1500000.0,21135RMFS,1999-12-10T00:00:00,Финансовые орга

In [5]:
AccruedInterestOnDate = RuDataDF("AccruedInterestOnDate").df
AccruedInterestOnDate.head()

AccruedInterestOnDate started
AccruedInterestOnDate finished. AccruedInterestOnDate shape (102155, 14)


,fintoolId,shiftDays,shiftTypeSource,typeCurrency,date,accruedInterest,accruedInterestPct,currentFaceValue,currentFaceValuePct,calculationType,error,faceValueCurrency,accruedInterestCurrency,faceValueLastKnownDate
0,111275,0,Calendar,None,2024-12-31T00:00:00,NaN,NaN,100000.0,100.0,Typical,None,RUB,RUB,2024-12-31T00:00:00
1,139679,0,Calendar,None,2024-12-31T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,USD,USD,2024-12-31T00:00:00
2,7691,0,Calendar,None,2024-12-31T00:00:00,NaN,NaN,100.0,100.0,Typical,None,RUB,RUB,2024-12-31T00:00:00
3,153935,0,Calendar,None,2024-12-31T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,USD,USD,2024-12-31T00:00:00
4,226763,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '226763': Н...,None,None,None


In [6]:
RUPriceHistory = RuDataDF("RUPriceHistory").df
RUPriceHistory.shape

RUPriceHistory started
RUPriceHistory finished. RUPriceHistory shape (40744, 82)


(40744, 82)

In [7]:
ExchangeTree = RuDataDF("ExchangeTree").df
ExchangeTree.head()

ExchangeTree started
ExchangeTree finished. ExchangeTree shape (239, 10)


,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn
0,144,63,Нал,Cash,Наличная валюта,Cash,0,2020-12-28T17:03:09,239,1
1,270,63,RUDIP 2,RUDIP 2,RUDIP 2,RUDIP 2,1,2022-10-31T19:41:24,239,2
2,84,83,США,US,Макроиндикаторы США,USA macroindicators,1,2020-12-28T17:03:09,239,3
3,85,83,РФ,RU,Макроиндикаторы Россия,Russia macroindicators,2,2020-12-28T17:03:09,239,4
4,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,239,5


In [8]:
MoexSecurities = RuDataDF("MoexSecurities").df
MoexSecurities.head()

MoexSecurities started
MoexSecurities finished. MoexSecurities shape (206391, 24)


,id_iss,secid,boardid,board_title,board_group,board_group_title,market,engine,short_name,isin,cfi_ifx,cfi_name,lot_size,expiration_date,currency,emitent_code,emitent_name,market_sector,min_price_increment,fintoolid,reg_code,update_date,counter,rn
0,2501,EUR_RUB__TOM,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Евро/Рубль-Том,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2021-02-28T23:56:16.686054,206391,1
1,2502,EUR_RUB__TOD,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Евро/Рубль-Тод,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2021-02-28T23:56:16.686053,206391,2
2,2840,KZT000000TOM,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Тенге/Рубль-Тод,None,YCRXXX,Валютные пары / Тип пары / Кросс-курс,10000,None,RUB,None,None,None,0.0025,NaN,None,2023-03-23T18:16:31.385678,206391,3
3,3691,UAH000000TOM,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Гривна/Рубль-Тод,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2018-09-17T09:17:51.410715,206391,4
4,3752,USD000000TOD,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Доллар/Рубль-Тод,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2021-02-28T23:56:17.686097,206391,5


In [9]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI

# isins = Prices[Prices["Price"].isna()].index.unique().tolist()
# key = "EndOfDay"
# url: str = getattr(DocsAPI, key).url
# payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
# payloads = []
# for isin in isins:
#     payload["isin"] = isin
#     payloads.append(payload.copy())
    
# async def end_of_day_rudata():
#     end_of_day = []
#     async with aiohttp.ClientSession(
#             connector=aiohttp.TCPConnector(limit=DocsAPI.LIMIT,
#                                            family=socket.AF_INET),
#             trust_env=True,
#             timeout=aiohttp.ClientTimeout(3600)
#     ) as session:
#         for i, chunk in enumerate(divide_chunks(payloads, DocsAPI.LIMIT)):
#             tasks: List[asyncio.Task] = [asyncio.create_task(
#                 RuDataRequest(url, session).post(payload=payload)
#             ) for payload in chunk]
#             resAll: List[List[dict]] = [await task for task in asyncio.as_completed(tasks, timeout=600)]
#             result: List[dict] = [row for task_res in resAll for row in task_res]
#             end_of_day.extend(result)
#     return end_of_day
    
# end_of_day = asyncio.run(end_of_day_rudata())
# EndOfDay = pd.DataFrame(end_of_day)
EndOfDay: pd.DataFrame = pd.read_sql(
                    f"""
                    SELECT *
                    FROM "EndOfDay"
                    where report_monthyear = '{RuDataDF.report_monthyear}'
                    """
                    , RuDataDF.engine
                )
EndOfDay.head()

CPU times: user 34.2 ms, sys: 3.17 ms, total: 37.4 ms
Wall time: 52.2 ms


,accruedint,admquote,akm_bl_n,akm_e_n,ask,ask_size,ask_size_total,avge_chng,avge_chng_pct,avge_prce,avge_yield,bid,bid_size,bid_size_total,boardid,boardname,buybackdate,buybackprice,cbr_lombard,cbr_pledge,chng,chng_m,chng_m_pct,chng_pct,chng_w,chng_w_pct,chng_y,chng_y_pct,close,consensus_num_est,consensus_time,convexity,convexity_agg,convexity_o,cpn,cpn_date,cpn_period,cpprice,deal_acc,discount,dnt06beg,dnt06max,dnt06min,dnt120beg,dnt120max,dnt120min,dnt14beg,dnt14max,dnt14min,dnt200beg,dnt200max,dnt200min,dnt365beg,dnt365max,dnt365min,duration,duration_agg,duration_n,duration_o,duration_on,exch,exp_bl_n,exp_e_n,facevalue,facevalue_currency,fact_frcst_chng,fch_b_i,fch_b_n,fch_e_dlti,fch_e_lti,forecast,frcst_chng,high,ifx_b_i,ifx_b_n,ifx_e_i,ifx_e_n,isin,issuer,issuesize,last,last_time,last_yield,last2avge,lastsettlecode,lclose,lnkcurrency,lotsize,low,mat_date,mat_date_dt,mcap,mds_b_i,mds_b_n,mds_e_lti,mds_e_ltn,mprice,mprice2,name,open,...,rec_buy_pct,rec_change,rec_hold_pct,rec_sell_pct,rec_val,recommendation,rus_b_n,rus_e_i,rus_e_n,seccode,secname,secname_e,sector,settlecode,shortname_eng,shortname_rus,snp_b_i,snp_b_n,snp_e_lti,snp_e_n,spread,status,target_price,tick_acc,time,tp_chng,tp_chng_m,tp_chng_m_pct,tp_chng_pct,tp_chng_q,tp_chng_q_pct,tp_chng_time,tp_chng_y,tp_chng_y_pct,tp_deviation,tp_high,tp_low,tp_median,val,val_acc,val_acc_usd,val_usd,vol,vol_acc,vol_lots,y2o_ask,y2o_bid,y2o_change,y2o_change_pct,y2o_last,y2o_prev,y2o_wap,yield_2,yield_agg,yield_ask,yield_average_12m,yield_average_1m,yield_average_2m,yield_average_2w,yield_average_3m,yield_average_6m,yield_average_o12m,yield_average_o1m,yield_average_o2m,yield_average_o2w,yield_average_o3m,yield_average_o6m,yield_bid,yield_chng,yield_cpprice,yield_cwa,yield_high,yield_low,yield_o_cwa,yield_o_swa,yield_open,yield_prev,yield_swa,total_return,full_price_pct,zspread,gspread,currency,fintoolId,id_iss,lcurrentprice,prevlclose,lcp_chng,lcp_chng_pct,bid_price_change,ask_price_change,bid_yield_change,ask_yield_change,id_trade_site,ifrs_level,rd_exch,rd_score,add_date,update_date,report_monthyear
0,40.85,None,None,None,101.99,None,34.0,0.47,0.46,101.67,20.037326,101.01,None,320.0,TQCB,Т+: Корп.облигации - безадрес.,2025-04-18T03:00:00,100.0,NaN,None,0.50,1.64,1.64,0.49,-1.28,-1.24,3.37,3.43,101.49,None,None,0.912382,0.287741,0.287741,52.36,2025-01-17T00:00:00,91.0,None,10.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,267.0,107.0,0.612098,107.0,0.254004,MICEX,None,None,1000.00,None,None,None,None,None,None,None,None,101.99,None,None,None,None,RU000A0JVWB3,None,10000000.0,101.99,2024-12-28T18:18:41,19.542657,0.79,None,None,RUB,1.0,101.20,17.10.2025,2025-10-17T00:00:00,1.016700e+09,None,None,None,None,101.83,None,РСХБ БО9.T.MICEX,101.21,...,None,None,None,None,None,None,None,None,None,RU000A0JVWB3,"""Россельхозбанк"" (АО) БО-09",Rosselkhozbank BO-09,None,None,None,None,None,None,None,None,0.98,None,NaN,None,2024-12-28T00:00:00,None,None,None,None,None,None,None,None,None,None,None,None,None,310103.0,310103.0,3084.68,3084.68,305.0,305.0,305.0,14.918076,18.631536,NaN,NaN,14.918076,NaN,16.113781,None,14.918076,19.542657,17.389602,14.044592,19.558989,12.693664,22.308970,22.098958,20.823538,12.521108,18.325395,10.987686,20.823538,20.823538,21.069175,NaN,None,20.655060,20.77051,19.542657,20.65506,14.666783,20.754829,NaN,17.891779,None,106.0750,None,None,RUB,100282,1112135,None,None,None,None,None,None,None,None,170,NaN,None,None,2024-12-28T23:55:34,2024-12-28T23:55:34,122024
1,NaN,None,None,None,1375.00,None,0.0,21.90,1.62,1371.40,NaN,1373.80,None,0.0,TQBR,T+ Акции и ДР,None,NaN,NaN,None,30.80,29.60,2.20,2.30,-167.40,-10.87,835.80,155.87,1341.20,None,None,NaN,NaN,NaN,NaN,None,NaN,None,24121.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,MICEX,None,None,0.67,None,None,None,None,None,None,None,None,1385.00,None,None,None,None,RU000A0JVW89,None,22977000.0,1372.00,2024-12-30T23:49:58,NaN,22.50,Non

In [10]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI
from time import sleep

# currencies = (
#     pd.read_sql(
#         """
#         SELECT DISTINCT currency
#         FROM currencies
#         """
#         , RuDataDF.engine
#     )['currency']
#     .to_list()
# )
# key = "CurrencyRate"
# url: str = getattr(DocsAPI, key).url
# payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
# currencies_rudata = []
# for currency in currencies:
#     payload["from"] = currency
#     ans = requests.post(url, json={'from': currency, 'to': 'RUB', 'date': REPORT_DATE_STR}, headers=RuDataRequest.headers)
#     result: dict = ans.json()
#     result["currency"] = currency
#     currencies_rudata.append(result)
#     sleep(1)
# CurrencyRate = pd.DataFrame(currencies_rudata)
CurrencyRate: pd.DataFrame = pd.read_sql(
                    f"""
                    SELECT *
                    FROM "CurrencyRate"
                    where report_monthyear = '{RuDataDF.report_monthyear}'
                    """
                    , RuDataDF.engine
                )
CurrencyRate.head()

CPU times: user 2.13 ms, sys: 1.16 ms, total: 3.29 ms
Wall time: 5.84 ms


,rate,endOfDayRate,error,currency,report_monthyear
0,19.842100,19.842100,None,LTL,122024
1,2.979890,2.979890,None,THB,122024
2,NaN,NaN,None,CDF,122024
3,13.122500,13.122500,None,HKD,122024
4,0.006273,0.006273,None,IDR,122024


##### Шаг 1.1. Discounts

###### Шаг 1.1.1. CBR

In [11]:
from sources.CBR_Soap import CBR_Soap

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.6834.111 Safari/537.36', 
           'Accept-Encoding': 'gzip, deflate', 
           'Accept': '*/*', 
           'Connection': 'keep-alive'}
cbr_disc = CBR_Soap()\
.get_discounts(date=last_work_date_month_str)[['ISIN', 'от 1 месяца до 180 дней_Beg']]\
.rename(columns={'от 1 месяца до 180 дней_Beg': 'CBR discount 6D'})\
.set_index('ISIN')

GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using urllib2 3.8
GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using urllib2 3.8
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using urllib2 3.8
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using urllib2 3.8


In [12]:
Prices.update(cbr_disc)
Prices['CBR discount 6D'] = Prices['CBR discount 6D'].fillna(1)

###### Шаг 1.1.2. nationalclearingcentre Рыночные риски

In [13]:
currMarketRatesResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/fondMarketRates?action=xls&xls=1&lang=ru&date={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)].reset_index(drop=True)
currMarketRates = currMarketRates\
.merge(MoexSecurities[['secid', 'isin']], how='left', left_on='Инструмент', right_on='secid')\
.rename(columns={'isin': 'ISIN'})\
.groupby('ISIN')\
.last()
currMarketRates = currMarketRates[currMarketRates.index.str.match('([A-Z]{2})([A-Z0-9]{9})([0-9]{1})$')]
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)]
currMarketRates=currMarketRates[['Ставка риска падения цены,%']]\
.rename(columns={'Ставка риска падения цены,%': 'Risk rate'})
currMarketRates['Discount NCC'] = currMarketRates['Risk rate'].apply(lambda x: min(30, ceil(x/np.sqrt(2)))) #Формула из п.12.1 Методики НКЦ определения риск-параметров
currMarketRates /= 100

In [14]:
Prices.update(currMarketRates)
Prices['Discount NCC'] = Prices['Discount NCC'].fillna(1)

###### Шаг 1.1.3. nationalclearingcentre Параметры ценных бумаг

In [15]:
securInfoResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/securInfo?action=xls&lang=ru&settleDate={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\
.rename(columns={
    'ISIN ЦБ': 'ISIN',
    'Принимается в обеспеч.': 'NCC collateral'
})\
.dropna()\
.set_index('ISIN')
securInfo = securInfo[securInfo['NCC collateral']=='Да']

In [16]:
Prices.update(securInfo)
Prices['NCC collateral'] = Prices['NCC collateral'].fillna('Нет')

##### Шаг 1.2. RuData transform

In [17]:
TransformFintoolReferenceData = FintoolReferenceData[['isincode', 'nickname', 'fintooltype', 'fintoolid', 'faceftname']]\
.rename(columns={
    "isincode": "ISIN",
    "nickname": "Name",
    "fintoolid": "Fintool",
    "fintooltype": "Type", 
    "faceftname": "Currency"
})
TransformFintoolReferenceData["ISIN in DB"] = 1
TransformFintoolReferenceData["Source"] = 'RU Data (ЦЦ НРД)'
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    AccruedInterestOnDate[['fintoolId', 'accruedInterest', 'currentFaceValue']], how="left", left_on="Fintool", right_on="fintoolId")
TransformFintoolReferenceData = TransformFintoolReferenceData\
.rename(columns={
    'accruedInterest': 'Interest', 
    'currentFaceValue': 'Principal'})\
.drop(columns=["fintoolId"])

In [18]:
RUPriceHistoryLast = RUPriceHistory.groupby('isincode').last().rename(columns={
    'vp_pct': 'Clean price pct', 
    'vp_pc': 'Clean price',
    'acc_int': 'InterestHistory'
}).reset_index()

In [19]:
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    RUPriceHistoryLast[['isincode', 'Clean price pct', 'Clean price', 'InterestHistory']], how='left', left_on="ISIN", right_on="isincode"
).drop(columns=["isincode"])
TransformFintoolReferenceData["Interest"] = TransformFintoolReferenceData["Interest"].combine_first(TransformFintoolReferenceData["InterestHistory"])
TransformFintoolReferenceData["Interest"].fillna(0, inplace=True)
TransformFintoolReferenceData.drop(columns="InterestHistory", inplace=True)

In [20]:
bonds_index = TransformFintoolReferenceData[TransformFintoolReferenceData["Type"]=='Облигация'].index

In [21]:
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"] + \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Interest"]

TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"]

In [22]:
Prices.update(TransformFintoolReferenceData.groupby("ISIN").first())

In [23]:
exch_prices = EndOfDay.merge(
    ExchangeTree[['id', 'fullname_rus']], how='left', left_on='id_trade_site', right_on='id')[['fintoolId', 'last', 'fullname_rus']]
exch_prices["Source"] = exch_prices['fullname_rus'].apply(lambda x: 'RU Data (' + str(x) + ')')
exch_prices = exch_prices\
.rename(columns={
    'fintoolId': 'Fintool',
    'last': 'PriceEX'
})\
.drop(columns="fullname_rus")
exch_prices = exch_prices.merge(TransformFintoolReferenceData[["Fintool", "ISIN", "Type", "Principal", "Interest"]], how='inner')

In [24]:
exch_bonds = exch_prices[exch_prices['Type']=='Облигация']
exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
exch_bonds["Price"] = exch_bonds["Clean price"] + exch_bonds["Interest"]

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_8635/1720937637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_8635/1720937637.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_8635/1720937637.py:4: SettingWithCopyWarning: 
A value is

In [25]:
exch_not_bonds = exch_prices[exch_prices['Type']!='Облигация']
exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
exch_not_bonds['Price'] = exch_not_bonds['PriceEX']

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_8635/3772375966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_8635/3772375966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Price'] = exch_not_bonds['PriceEX']


In [26]:
Prices.update(exch_bonds.set_index("ISIN"))
Prices.update(exch_not_bonds.set_index("ISIN"))

In [27]:
ResultPrices = Prices[~Prices['Price'].isna()]
Prices = Prices[Prices['Price'].isna()]

#### Шаг 2. Cbonds

##### Шаг 2.1. Bonds

In [28]:
tradings_bonds = pd.read_excel(f'./data/Input/tradings_bonds.xls')
tradings_bonds_1 = pd.read_excel(f'./data/Input/tradings_bonds_1.xls')

tradings_bonds = pd.concat([tradings_bonds, tradings_bonds_1], axis=0)
tradings_bonds = tradings_bonds[['ISIN', 'Краткое название эмитента (rus)', 'Валюта', 'Индикативная цена', 'Номинал по еврооблигациям']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта': 'Currency', 
    'Индикативная цена': 'Clean price pct', 
    'Номинал по еврооблигациям': 'Principal EB'
})

In [29]:
tradings_bonds = tradings_bonds.merge(Prices[["Interest", "Principal"]], how='inner', left_on="ISIN", right_index=True)
principal_idx = tradings_bonds[tradings_bonds['Principal']>tradings_bonds['Principal EB']].index
tradings_bonds.loc[principal_idx, 'Principal'] /= tradings_bonds.loc[principal_idx, 'Principal EB']
tradings_bonds.loc[principal_idx, 'Interest'] /= tradings_bonds.loc[principal_idx, 'Principal']

In [30]:
Prices.update(tradings_bonds.set_index("ISIN"))

In [31]:
tradings_nkd = pd.read_excel(f'./data/Input/tradings_nkd.xls')
tradings_nkd_1 = pd.read_excel(f'./data/Input/tradings_nkd_1.xls')
tradings_nkd_2 = pd.read_excel(f'./data/Input/tradings_nkd_2.xls')
tradings_nkd_3 = pd.read_excel(f'./data/Input/tradings_nkd_3.xls')

tradings_nkd = pd.concat([tradings_nkd, tradings_nkd_1, tradings_nkd_2, tradings_nkd_3], axis=0)
tradings_nkd = tradings_nkd[['ISIN код эмиссии', 'Накопленный купонный доход', 'Непогашенный номинал']]\
.rename(columns={
    'ISIN код эмиссии': "ISIN", 
    'Накопленный купонный доход': "Interest", 
    'Непогашенный номинал': "Principal"
})
tradings_nkd = tradings_nkd.groupby("ISIN").first()

In [32]:
Prices.update(tradings_nkd)

In [33]:
tradings_new = pd.read_excel(f'./data/Input/tradings_new_{REPORT_DATE_STR}.xls')

tradings_new = tradings_new[['Код ISIN', 'Эмиссия', 'Дата', 'Bid, закр.', 'НКД']]\
.rename(columns={
    'Код ISIN': 'ISIN',
    'Эмиссия': 'Name',
    'Дата': 'Date',
    'Bid, закр.': 'Clean price pct',
    'НКД': 'Interest'
})
tradings_new["Max date"] = tradings_new.groupby('ISIN')["Date"].transform('max')
tradings_new = tradings_new.groupby('ISIN').first()

In [34]:
emissions = pd.read_excel(f'./data/Input/emissions_{REPORT_DATE_STR}.xls')

emissions = emissions[['Валюта', 'код ISIN', 'Непогашенный номинал']]\
.rename(columns={
    'код ISIN': 'ISIN',
    'Валюта': 'Currency',
    'Непогашенный номинал': 'Principal'
})
emissions = emissions[~((emissions["ISIN"].isna()) | (emissions["Principal"].isna()) | (emissions["Principal"]==0))].set_index("ISIN")
tradings_new = tradings_new.merge(emissions, how="left", left_index=True, right_index=True)

In [35]:
Prices.update(tradings_new)

In [36]:
Prices.loc[Prices['Interest'] > 0.2*Prices['Principal'], 'Interest'] = 0
Prices['Clean price'] = Prices['Clean price pct']/100 * Prices['Principal']
Prices['Price'] = Prices['Clean price'] + Prices['Interest']

In [37]:
cbonds_bonds_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[cbonds_bonds_idx, "Type"] = 'Облигация'
Prices.loc[cbonds_bonds_idx, 'Source'] = 'Cbonds'
Prices.loc[cbonds_bonds_idx, 'ISIN in DB'] = 1

In [38]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[cbonds_bonds_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

##### Шаг 2.2. Stocks

In [39]:
tradings_stocks_all = pd.read_excel(f'./data/Input/tradings_stocks_all.xls')

tradings_stocks_all = tradings_stocks_all[['ISIN', 'Краткое название эмитента (rus)', 'Валюта номинала', 'Валюта торгов', 'Цена закрытия']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта номинала': 'Currency', 
    'Валюта торгов': 'Trade currency', 
    'Цена закрытия': 'Price'
})

In [40]:
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Trade currency", right_on="currency")\
.rename(columns={'rate': 'trade rate'})
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Currency", right_on="currency")
tradings_stocks_all['RUB Price'] = tradings_stocks_all['Price'] * tradings_stocks_all['trade rate']
tradings_stocks_all['RUB Price'] = tradings_stocks_all.groupby("ISIN")['RUB Price'].transform('mean')
tradings_stocks_all['Currency'] = tradings_stocks_all['Currency'].combine_first(tradings_stocks_all['Trade currency'])
tradings_stocks_all['Clean price'] = tradings_stocks_all['RUB Price'] / tradings_stocks_all['rate']
tradings_stocks_all['Price'] = tradings_stocks_all['Clean price']
tradings_stocks_all['FX Rate'] = tradings_stocks_all['trade rate'].combine_first(tradings_stocks_all['rate'])

In [41]:
cbonds_stocks_idx = tradings_stocks_all[~tradings_stocks_all["Price"].isna()].index
tradings_stocks_all.loc[cbonds_stocks_idx, "Type"] = 'Акция'
tradings_stocks_all.loc[cbonds_stocks_idx, 'Source'] = 'Cbonds'
tradings_stocks_all.loc[cbonds_stocks_idx, 'ISIN in DB'] = 1

In [42]:
Prices.update(tradings_stocks_all.groupby("ISIN").first())

In [43]:
ResultPrices = pd.concat([ResultPrices, Prices[~Prices["Price"].isna()]], axis=0)
Prices = Prices[Prices['Price'].isna()]

In [44]:
Prices.shape

(1811, 16)

#### Шаг 3. MOEX

##### Шаг 3.0. RuData MOEX stage

In [47]:
HistoryStockBonds = RuDataDF("HistoryStockBonds").df
HistoryStockBonds.head()

HistoryStockBonds started
HistoryStockBonds finished. HistoryStockBonds shape (118678, 66)


,boardname,regnumber,isin,matdate,listname,facevalue,currencyid,volume,value,numtrades,prev,prevlegalcloseprice,openperiod,legalopenprice,legalcloseprice,closeperiod,waprice,trendclose,trendclspr,trendwap,trendwappr,openval,closeval,highbid,lowoffer,bid,offer,yieldatwap,yieldclose,accint,duration,marketprice,marketprice2,admittedquote,issuesize,mpvaltrd,mp2valtrd,admittedvalue,marketprice3,marketprice3tradesvalue,type,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Т+: Облигации - безадрес.,12840108V,RU000A10A810,2026-05-27T00:00:00,1,200000.0,SUR,29,5.508726e+08,10,95.00,95.000,NaN,None,95.750,None,95.7155,0.75,0.79,0.5741,0.60,37905669.00,19002449.25,99.50,95.50,95.5001,100.00,8.19,8.16,78557.0183,502.0,None,95.7155,None,15000,0.0,5.508726e+08,None,95.7155,5.508726e+08,ofz_bond,95.75,NaN,NaN,4.75,4750.0,None,2024-12-27T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,99.50,95.50,95.50,95.75,RU000A10A810,РФ ЗО 26 Д,2024-12-27T00:00:00,1,NaN
1,Т+: Облигации - безадрес.,12840108V,RU000A10A810,2026-05-27T00:00:00,1,200000.0,SUR,0,0.000000e+00,0,95.75,95.750,NaN,None,95.750,None,NaN,NaN,NaN,NaN,NaN,0.00,0.00,95.00,97.00,95.0000,100.00,NaN,NaN,82238.0175,NaN,None,95.7155,None,15000,0.0,5.508726e+08,None,95.7155,5.508726e+08,ofz_bond,NaN,NaN,NaN,4.75,4750.0,None,None,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,NaN,NaN,NaN,NaN,RU000A10A810,РФ ЗО 26 Д,2024-12-28T00:00:00,1,NaN
2,Т+: Облигации - безадрес.,12840109V,RU000A10A851,2027-06-23T00:00:00,1,200000.0,SUR,0,0.000000e+00,0,NaN,NaN,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,0.00,0.00,74.01,94.99,74.0100,94.99,NaN,NaN,398703.0825,NaN,None,NaN,None,12023,0.0,0.000000e+00,None,NaN,0.000000e+00,ofz_bond,NaN,NaN,NaN,4.25,4250.0,None,None,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,NaN,NaN,NaN,NaN,RU000A10A851,РФ ЗО 27 Д,2024-12-05T00:00:00,3,NaN
3,Т+: Облигации - безадрес.,12840109V,RU000A10A851,2027-06-23T00:00:00,1,200000.0,SUR,2,3.463147e+07,2,NaN,NaN,NaN,None,83.745,None,83.2925,NaN,NaN,NaN,NaN,17366393.93,17265077.90,83.99,83.50,82.0000,84.00,12.46,12.34,397883.5417,863.0,None,NaN,None,12023,0.0,0.000000e+00,None,NaN,0.000000e+00,ofz_bond,83.50,NaN,NaN,4.25,4250.0,None,2024-12-06T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,83.99,83.50,83.99,83.50,RU000A10A851,РФ ЗО 27 Д,2024-12-06T00:00:00,3,NaN
4,Т+: Облигации - безадрес.,12840109V,RU000A10A851,2027-06-23T00:00:00,1,200000.0,SUR,1,1.681416e+07,1,83.50,83.745,NaN,None,84.560,None,84.5600,1.06,1.27,1.2675,1.52,16814164.08,16814164.08,88.00,84.56,86.1100,89.99,11.77,11.77,389676.6040,863.0,None,NaN,None,12023,0.0,0.000000e+00,None,NaN,0.000000e+00,ofz_bond,84.56,NaN,NaN,4.25,4250.0,None,2024-12-09T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,84.56,84.56,84.56,84.56,RU000A10A851,РФ ЗО 27 Д,2024-12-09T00:00:00,3,NaN


In [48]:
HistoryStockShares = RuDataDF("HistoryStockShares").df
HistoryStockShares.head()

HistoryStockShares started
HistoryStockShares finished. HistoryStockShares shape (26722, 54)


,boardname,regnumber,isin,listname,facevalue,currencyid,prevlegalcloseprice,prev,legalopenprice,openperiod,closeperiod,openval,closeval,legalcloseprice,trendclose,trendclspr,highbid,bid,offer,lowoffer,waprice,trendwap,trendwappr,volume,marketprice,marketprice2,mp2valtrd,mpvaltrd,value,numtrades,issuesize,admittedquote,admittedvalue,monthlycapitalization,dailycapitalization,marketprice3,marketprice3tradesvalue,type,closeauctionprice,waval,lastprice,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Т+: Акции и ДР - безадрес.,2-01-55072-E,RU000A0ET5B9,3,11.88,SUR,1116.0,1116.0,None,NaN,None,1110.0,12584.0,1128.0,28.0,2.51,1144.0,1106.0,1142.0,1110.0,1126.0,-2.0,-0.18,54,None,NaN,0.0,0.0,60758.0,15,1063512,None,None,1.293231e+09,1.197515e+09,1128.0,502258.0,preferred_share,1128.0,0.0,1144.0,stock,shares,TQBR,0,1144.0,1110.0,1110.0,1144.0,NNSBP,ТНСэнНН ап,2024-12-13T00:00:00,1,NaN
1,Т+: Акции и ДР - безадрес.,2-01-55072-E,RU000A0ET5B9,3,11.88,SUR,1128.0,1144.0,None,NaN,None,1144.0,5460.0,1092.0,-52.0,-4.55,1120.0,1090.0,1092.0,1092.0,1106.0,-20.0,-1.78,44,None,NaN,0.0,0.0,48642.0,29,1063512,None,None,1.293231e+09,1.176244e+09,1124.0,500146.0,preferred_share,1092.0,0.0,1092.0,stock,shares,TQBR,0,1144.0,1092.0,1144.0,1092.0,NNSBP,ТНСэнНН ап,2024-12-16T00:00:00,1,NaN
2,Т+: Акции и ДР - безадрес.,2-01-55072-E,RU000A0ET5B9,3,11.88,SUR,1092.0,1092.0,None,NaN,None,1106.0,3120.0,1040.0,-52.0,-4.76,1106.0,1038.0,1040.0,1040.0,1070.0,-36.0,-3.25,239,None,NaN,0.0,0.0,255642.0,64,1063512,None,None,1.293231e+09,1.137958e+09,1096.0,518234.0,preferred_share,1040.0,0.0,1040.0,stock,shares,TQBR,0,1120.0,1040.0,1106.0,1040.0,NNSBP,ТНСэнНН ап,2024-12-17T00:00:00,1,NaN
3,Т+: Акции и ДР - безадрес.,2-01-55072-E,RU000A0ET5B9,3,11.88,SUR,1040.0,1040.0,None,NaN,None,1034.0,2124.0,1062.0,22.0,2.12,1066.0,1064.0,1066.0,1026.0,1046.0,-24.0,-2.24,241,None,NaN,0.0,0.0,252224.0,54,1063512,None,None,1.293231e+09,1.112434e+09,1058.0,504564.0,preferred_share,1062.0,0.0,1064.0,stock,shares,TQBR,0,1070.0,1032.0,1034.0,1062.0,NNSBP,ТНСэнНН ап,2024-12-18T00:00:00,1,NaN
4,Т+: Акции и ДР - безадрес.,2-01-55072-E,RU000A0ET5B9,3,11.88,SUR,1062.0,1062.0,None,NaN,None,2132.0,3270.0,1090.0,28.0,2.64,1126.0,1090.0,1104.0,1032.0,1104.0,58.0,5.54,82,None,NaN,0.0,0.0,90530.0,38,1063512,None,None,1.293231e+09,1.174117e+09,1058.0,502668.0,preferred_share,1090.0,0.0,1090.0,stock,shares,TQBR,0,1130.0,1066.0,1066.0,1090.0,NNSBP,ТНСэнНН ап,2024-12-19T00:00:00,1,NaN


In [49]:
HistoryStockNdm = RuDataDF("HistoryStockNdm").df
HistoryStockNdm.head()

HistoryStockNdm started
HistoryStockNdm finished. HistoryStockNdm shape (25397, 47)


,boardname,closeval,currencyid,facevalue,isin,issuesize,listname,matdate,numtrades,openval,prev,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,РПС с ЦК: Облигации - адрес.,0.0,SUR,100000.0,RU000A0ZZVE6,17500,1,2025-12-04T00:00:00,0,0.0,91.55,12978082V,NaN,NaN,NaN,NaN,ofz_bond,0.0,0,NaN,NaN,NaN,None,None,None,2.88,2875.0,None,2024-10-30T00:00:00,None,NaN,NaN,None,EUR,stock,ndm,PTOB,4,NaN,NaN,NaN,NaN,RU000A0ZZVE6,RUS-25 EUR,2024-12-26T00:00:00,2,NaN
1,РПС с ЦК: Облигации (EUR) - адрес.,0.0,EUR,100000.0,RU000A0ZZVE6,17500,1,2025-12-04T00:00:00,0,0.0,97.05,12978082V,NaN,NaN,NaN,NaN,ofz_bond,0.0,0,NaN,NaN,NaN,None,None,None,2.88,2875.0,None,2024-06-06T00:00:00,None,NaN,NaN,None,EUR,stock,ndm,PTOE,4,NaN,NaN,NaN,NaN,RU000A0ZZVE6,RUS-25 EUR,2024-12-26T00:00:00,2,NaN
2,РПС: Облигации (EUR) - адрес.,0.0,EUR,100000.0,RU000A0ZZVE6,17500,1,2025-12-04T00:00:00,0,0.0,97.85,12978082V,NaN,NaN,NaN,NaN,ofz_bond,0.0,0,NaN,NaN,NaN,None,None,None,2.88,2875.0,None,2022-06-29T00:00:00,None,NaN,NaN,None,EUR,stock,ndm,PSEO,4,NaN,NaN,NaN,NaN,RU000A0ZZVE6,RUS-25 EUR,2024-12-27T00:00:00,2,NaN
3,РПС: Облигации - адрес.,0.0,SUR,100000.0,RU000A0ZZVE6,17500,1,2025-12-04T00:00:00,0,0.0,95.00,12978082V,NaN,NaN,NaN,NaN,ofz_bond,0.0,0,NaN,NaN,NaN,None,None,None,2.88,2875.0,None,2024-10-23T00:00:00,None,NaN,NaN,None,EUR,stock,ndm,PSOB,4,NaN,NaN,NaN,NaN,RU000A0ZZVE6,RUS-25 EUR,2024-12-27T00:00:00,2,NaN
4,РПС с ЦК: Облигации - адрес.,0.0,SUR,100000.0,RU000A0ZZVE6,17500,1,2025-12-04T00:00:00,0,0.0,91.55,12978082V,NaN,NaN,NaN,NaN,ofz_bond,0.0,0,NaN,NaN,NaN,None,None,None,2.88,2875.0,None,2024-10-30T00:00:00,None,NaN,NaN,None,EUR,stock,ndm,PTOB,4,NaN,NaN,NaN,NaN,RU000A0ZZVE6,RUS-25 EUR,2024-12-27T00:00:00,2,NaN


In [50]:
HistoryStockCcp = RuDataDF("HistoryStockCcp").df
HistoryStockCcp.head()

HistoryStockCcp started
HistoryStockCcp finished. HistoryStockCcp shape (34685, 54)


,accint,admittedquote,admittedvalue,bid,boardname,closeperiod,closeval,currencyid,duration,facevalue,highbid,isin,issuesize,legalcloseprice,legalopenprice,listname,lowoffer,marketprice,marketprice2,marketprice3,marketprice3tradesvalue,matdate,mp2valtrd,mpvaltrd,numtrades,offer,openperiod,openval,prev,prevlegalcloseprice,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,21.17,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,1.028696e+06,SUR,1944.0,1000.0,20.0,RU000A0JV4N8,190287001,None,None,1,18.00,20.34,None,None,0.0,2032-05-05T00:00:00,0.0,0.0,8,None,None,1.271201e+08,19.85,None,29009RMFS,-1.85,None,-0.32,NaN,ofz_bond,6.665322e+08,717916,19.49,None,None,stock,ccp,EQRP,4,20.0,18.00,20.0,18.0,SU29009RMFS6,ОФЗ 29009,2024-12-24T00:00:00,1,NaN
1,21.17,None,None,None,РЕПО с ЦК 7 дн. - безадрес.,None,2.366784e+09,SUR,1944.0,1000.0,20.1,RU000A0JV4N8,190287001,None,None,1,20.10,20.34,None,None,0.0,2032-05-05T00:00:00,0.0,0.0,1,None,None,2.366784e+09,20.20,None,29009RMFS,-0.10,None,-0.16,NaN,ofz_bond,2.366784e+09,2549242,20.10,None,None,stock,ccp,EQWP,4,20.1,20.10,20.1,20.1,SU29009RMFS6,ОФЗ 29009,2024-12-24T00:00:00,1,NaN
2,21.17,None,None,None,РЕПО с ЦК - адрес.,None,6.096460e+03,SUR,NaN,1000.0,NaN,RU000A0JV4N8,190287001,None,None,1,NaN,20.34,None,None,0.0,2032-05-05T00:00:00,0.0,0.0,37,None,None,1.219290e+03,NaN,None,29009RMFS,NaN,None,NaN,NaN,ofz_bond,2.298200e+05,211,NaN,None,None,stock,ccp,PSRP,4,NaN,NaN,NaN,NaN,SU29009RMFS6,ОФЗ 29009,2024-12-24T00:00:00,1,NaN
3,21.69,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,2.597009e+06,SUR,1943.0,1000.0,20.0,RU000A0JV4N8,190287001,None,None,1,19.46,20.40,None,None,0.0,2032-05-05T00:00:00,0.0,0.0,6,None,None,5.132428e+06,18.00,None,29009RMFS,1.80,None,0.44,NaN,ofz_bond,3.633377e+08,389359,19.93,None,None,stock,ccp,EQRP,4,20.0,19.46,20.0,19.8,SU29009RMFS6,ОФЗ 29009,2024-12-25T00:00:00,1,NaN
4,21.69,None,None,None,РЕПО с ЦК - адрес.,None,1.017960e+03,SUR,NaN,1000.0,NaN,RU000A0JV4N8,190287001,None,None,1,NaN,20.40,None,None,0.0,2032-05-05T00:00:00,0.0,0.0,32,None,None,1.221810e+03,NaN,None,29009RMFS,NaN,None,NaN,NaN,ofz_bond,2.087001e+05,191,NaN,None,None,stock,ccp,PSRP,4,NaN,NaN,NaN,NaN,SU29009RMFS6,ОФЗ 29009,2024-12-25T00:00:00,1,NaN


##### Шаг 3.1. Tranform

In [51]:
TransformHistoryStockBonds = HistoryStockBonds.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockShares = HistoryStockShares.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockNdm = HistoryStockNdm.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockCcp = HistoryStockCcp.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")

In [52]:
Prices.update(TransformHistoryStockBonds)
Prices.update(TransformHistoryStockShares)
Prices.update(TransformHistoryStockNdm)
Prices.update(TransformHistoryStockCcp)

In [53]:
moex_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[moex_idx, 'Source'] = 'RU Data (MOEX)'
Prices.loc[moex_idx, 'ISIN in DB'] = 1

In [54]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[moex_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

In [55]:
Prices.shape

(1810, 16)

#### Шаг 4. Finish

In [56]:
FullPrices = pd.concat([ResultPrices, Prices], axis=0)

##### Шаг 4.1. Update currency

In [57]:
FullPrices = FullPrices\
.reset_index()\
.merge(CurrencyRate[['currency', 'rate']], how='left', left_on='Currency', right_on='currency')\
.set_index("ISIN")

In [58]:
rub_price_idx = FullPrices[FullPrices["RUB Price"].isna()].index
FullPrices.loc[rub_price_idx, "FX Rate"] = FullPrices.loc[rub_price_idx, "rate"].fillna(1)
FullPrices.loc[rub_price_idx, "RUB Price"] = FullPrices.loc[rub_price_idx, "Price"] * FullPrices.loc[rub_price_idx, "FX Rate"]

In [59]:
FullPrices.drop(columns=['currency', 'rate'], inplace=True)

##### Шаг 4.2. Write excel

In [60]:
FullPrices['export_timestamp'] = dt.now().strftime("%Y%m%d%H%M%S")

In [61]:
with pd.ExcelWriter(f'./data/Output/prices.xlsx', datetime_format='dd.mm.yyyy') as writer: 
    FullPrices[~FullPrices["Price"].isna()].to_excel(writer)

In [62]:
with pd.ExcelWriter(f'./data/Output/prices_full.xlsx', datetime_format='dd.mm.yyyy') as writer: 
    FullPrices.to_excel(writer)

In [63]:
from functions.send_email import main
main("prices.xlsx", "prices_full.xlsx")